In [1]:
import gurobipy as gb
from gurobipy import *
import numpy as np
import pandas as pd

# Manchester United of 2016-2017 Europa League Season

## Part 1. Main Model - Select 4 new players

In [2]:
# Import the dataset
full_players_df=pd.read_csv('players_dataset.csv')

# Fill the blank value as 0
full_players_df.fillna(0, inplace=True)

# List of existing players
existing_players_names = ["David de Gea", "Eric Bailly", "Phil Jones", "Marcos Rojo", "Paul Pogba", "Memphis Depay", "Juan Mata", "Zlatan Ibrahimović", "Wayne Rooney", 
    "Anthony Martial", "Chris Smalling", "Jesse Lingard", "Adnan Januzaj", "Michael Carrick", "Daley Blind", "Ashley Young", "Marcus Rashford", 
    "Sergio Romero", "Ander Herrera"]

# Extract the rows for these players
filtered_players_df = full_players_df[~full_players_df['long_name'].isin(existing_players_names)]

# Convert the filtered DataFrame to a dictionary for use in the model
players = {row['player_id']: row.to_dict() for index, row in filtered_players_df.iterrows()}

# Create a dictionary to hold the potential positions for each player
player_positions = {player_id: 'Unassigned' for player_id in players.keys()}

# Tag each player with their potential position
for player_id, player in players.items():
    if 'GK' in player['player_positions']:
        player_positions[player_id] = 'Goalkeeper (GK)'
    elif 'CB' in player['player_positions'] or 'LB' in player['player_positions'] or 'RB' in player['player_positions']:
        player_positions[player_id] = 'Defender (including CB, LB, and DB)'
    elif 'CDM' in player['player_positions']:
        player_positions[player_id] = 'Defensive Midfielder (CDM)'
    elif 'CAM' in player['player_positions'] or 'LM' in player['player_positions'] or 'RM' in player['player_positions'] or 'LW' in player['player_positions'] or 'RW' in player['player_positions']:
        player_positions[player_id] = 'Attacking Midfielder/Winger (including CAM, LM/RM, LW/RW)'
    elif 'ST' in player['player_positions']:
        player_positions[player_id] = 'Striker (ST)'

In [3]:
# Initialize the model
model1 = gb.Model("Player_Selection")

# Add variables
x = model1.addVars(players.keys(), vtype=GRB.BINARY, name="x")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30


In [4]:
# Create a dictionary to hold the potential positions for each player
player_positions = {player_id: 'Unassigned' for player_id in players.keys()}

# Now when creating your objective functions, tag each player with their potential position
for player_id, player in players.items():
    if 'GK' in player['player_positions']:
        player_positions[player_id] = 'Goalkeeper (GK)'
    elif 'CB' in player['player_positions'] or 'LB' in player['player_positions'] or 'RB' in player['player_positions']:
        player_positions[player_id] = 'Defender (including CB, LB, and DB)'
    elif 'CDM' in player['player_positions']:
        player_positions[player_id] = 'Defensive Midfielder (CDM)'
    elif 'CAM' in player['player_positions'] or 'LM' in player['player_positions'] or 'RM' in player['player_positions'] or 'LW' in player['player_positions'] or 'RW' in player['player_positions']:
        player_positions[player_id] = 'Attacking Midfielder/Winger (including CAM, LM/RM, LW/RW)'
    elif 'ST' in player['player_positions']:
        player_positions[player_id] = 'Striker (ST)'

In [5]:
# Objective function
# Goalkeepers
obj_gk = quicksum(x[player_id] * (player['goalkeeping_diving'] + player['goalkeeping_handling'] +
                                  player['goalkeeping_positioning'] + player['goalkeeping_reflexes'] +
                                  player['goalkeeping_kicking'])
                 for player_id, player in players.items() if 'GK' in player['player_positions'])

# Defenders
obj_def = quicksum(x[player_id] * (player['defending_marking_awareness'] + player['power_strength'] +
                                   player['mentality_composure'] + player['movement_reactions'] +
                                   player['defending_standing_tackle'] + player['defending_sliding_tackle'])
                 for player_id, player in players.items() if 'CB' in player['player_positions'] or 'LB' in player['player_positions'] or 'RB' in player['player_positions'])

# Defensive Midfielders
obj_dmf = quicksum(x[player_id] * (player['defending'] + player['mentality_interceptions'] +
                                   player['passing'] + player['mentality_aggression'] +
                                   player['physic'] + player['power_stamina'] +
                                   player['mentality_positioning'])
                 for player_id, player in players.items() if 'CMD' in player['player_positions'])

# Attacking Midfielders/Wingers
obj_amf = quicksum(x[player_id] * (player['dribbling'] + player['attacking_crossing'] +
                                   player['skill_curve'] + player['attacking_short_passing'] +
                                   player['mentality_vision'] + player['movement_agility'])
                 for player_id, player in players.items() if 'CAM' in player['player_positions'] or 'LM' in player['player_positions'] or 'RM' in player['player_positions'] or 'LW' in player['player_positions'] or 'RW' in player['player_positions'])

# Strikers
obj_st = quicksum(x[player_id] * (player['shooting'] + player['attacking_finishing'] +
                                  player['pace'] + player['mentality_composure'] +
                                  player['attacking_heading_accuracy'] + player['skill_ball_control'] +
                                  player['movement_reactions'])
                 for player_id, player in players.items() if 'ST' in player['player_positions'])

obj_overall = quicksum(x[player_id] * player['overall'] for player_id, player in players.items())

model1.modelSense=GRB.MAXIMIZE
# Set the objectives with different priorities
model1.setObjectiveN(obj_dmf, index=3, priority=2, name='MaximizeDefensiveMidfielderSkills')
model1.setObjectiveN(obj_st, index=2, priority=3, name='MaximizeStrikerSkills')
model1.setObjectiveN(obj_amf, index=1, priority=4, name='MaximizeAttackingMidfielderSkills')
model1.setObjectiveN(obj_def, index=4, priority=1, name='MaximizeDefenderSkills')
model1.setObjectiveN(obj_gk, index=5, priority=0, name='MaximizeGoalKeepersSkills')
model1.setObjectiveN(obj_overall, index=0, priority=5, name='MaximizeOverallScore')

In [6]:
# Constraints

# 1. Transfer Budget
transfer_budget = 200e6  # In pounds
exchange_rate = 1.2

model1.addConstr(
    quicksum(x[player_id] * player['value_eur'] for player_id, player in players.items()) <= transfer_budget * exchange_rate,
    "Transfer Budget"
)

# 2. Wage Budget
revenue = 581e6  # In pounds
exchange_rate = 1.2
max_wage_percentage = 0.70
wage_budget = revenue * max_wage_percentage * exchange_rate
wage_budget_per_week = wage_budget / 52

model1.addConstr(
    quicksum(x[player_id] * player['wage_eur'] for player_id, player in players.items()) <= wage_budget_per_week,
    "wage_constraint"
)

# 3. Non-Euro Player
max_non_eu_players = 17
model1.addConstr(
    quicksum(x[player_id] for player_id, player in players.items() if player.get('non_eu', 0) == 1) <= max_non_eu_players,
    "non_eu_players"
)

# 4. Four Players
model1.addConstr(quicksum(x[player_id] for player_id in players.keys()) == 4, "Select_4_new_players")

# 5. Just One Goalkeeper
goalkeepers = [player_id for player_id, player in players.items() if 'GK' in player['player_positions']]
model1.addConstr(quicksum(x[gk] for gk in goalkeepers) == 1, "exactly_one_goalkeeper")

# 6. Age constraint
max_avg_age = 27
total_age = quicksum(x[player_id] * player['age'] for player_id, player in players.items())
total_players = quicksum(x[player_id] for player_id in players.keys())
model1.addConstr(total_age <= max_avg_age * total_players, "average_age")

<gurobi.Constr *Awaiting Model Update*>

In [7]:
# Optimize the model
model1.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 6 rows, 22901 columns and 92107 nonzeros
Model fingerprint: 0xddad405c
Variable types: 0 continuous, 22901 integer (22901 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [4e+01, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 6 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 2 rows and 0 columns
Presolve time: 0.04s
Presolved: 4

In [8]:
# Check if the optimization was successful
if model1.status == GRB.OPTIMAL:
    # Print the objective value for each objective function
    for i in range(model1.NumObj):  # Loop through each objective
        model1.setParam(GRB.Param.ObjNumber, i)
        print(f"Objective {i} value: {model1.ObjNVal}")
    # Calculate the used budget
    used_budget = sum(x[player_id].X * player['value_eur'] for player_id, player in players.items() if x[player_id].X > 0.5)
    print(f"Used Budget (In Euro): {used_budget}")

    # Extract and print selected player details
    selected_players_info = [(player_id, player['long_name'], player['player_positions']) 
                             for player_id, player in players.items() if x[player_id].X > 0.5]
    
    # Create a DataFrame to display selected player details neatly
    selected_players_df = pd.DataFrame(selected_players_info, columns=['Player ID', 'Long Name', 'Player Positions'])
    print(selected_players_df)

else:
    print(f"Optimization was not successful. Status code: {model1.status}")

Objective 0 value: 356.0
Objective 1 value: 513.0
Objective 2 value: 1228.0
Objective 3 value: 0.0
Objective 4 value: 529.0
Objective 5 value: 438.0
Used Budget (In Euro): 239500000.0
   Player ID                            Long Name Player Positions
0      20801  Cristiano Ronaldo dos Santos Aveiro           LW, ST
1     193080                David De Gea Quintana               GK
2     138956                    Giorgio Chiellini               CB
3     231747                 Kylian Mbappé Lottin               ST


## Part 2. Starting Line-Up Squad Selection (For 4-2-3-1 Formation)

In [9]:
# Load data
team = pd.read_csv('starting_lineup.csv')

# Create optimization model
model2 = gb.Model("Football Team Selection 4-2-3-1")

# Set dummy variables 
all_positions = set()
for positions in team['player_positions']:
    all_positions.update(positions.split(", "))

for position in all_positions:
    team[f'{position}_dummy'] = team['player_positions'].apply(lambda x: 1 if position in x else 0)

# Set decision variables: whether to select a player for a specific position
select = model2.addVars([(p, pos) for p in team.index for pos in all_positions], vtype=GRB.BINARY, name="select")

In [10]:
# Objective functions

# Objective function for goalkeeper
obj_goalkeeper = gb.quicksum(select[p, 'GK'] * sum(team.loc[p, ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_kicking']]) 
                            for p in team.index if team.loc[p, 'GK_dummy'] == 1)


# Objective function for defenders (CB, LB, RB)
obj_def = gb.quicksum(select[p, pos] * sum(team.loc[p, ['defending_marking_awareness', 'power_strength', 'mentality_composure', 
                                                       'movement_reactions', 'defending_standing_tackle', 'defending_sliding_tackle']])
                     for p in team.index 
                     for pos in ['CB', 'LB', 'RB'] 
                     if team.loc[p, f'{pos}_dummy'] == 1)

    
# Objective function for central defensive midfielders (CDM)
obj_def_mid = gb.quicksum(select[p, 'CDM'] * sum(team.loc[p, ['defending', 'mentality_interceptions', 'passing', 
                                                              'mentality_aggression', 'physic', 'power_stamina', 
                                                              'mentality_positioning']])
                         for p in team.index if team.loc[p, 'CDM_dummy'] == 1)

# Objective function for attacking midfielders (CAM, LM, RM, LW)
obj_att_mid = gb.quicksum(select[p, pos] * sum(team.loc[p, ['dribbling', 'attacking_crossing', 'skill_curve', 
                                                            'attacking_short_passing', 'mentality_vision', 
                                                            'movement_agility']])
                         for p in team.index 
                         for pos in ['CAM', 'LM', 'RM', 'LW'] 
                         if team.loc[p, f'{pos}_dummy'] == 1)


# Objective function for strikers (ST)
obj_striker = gb.quicksum(select[p, 'ST'] * sum(team.loc[p, ['shooting', 'pace', 'mentality_composure', 
                                                             'attacking_heading_accuracy', 'skill_ball_control', 
                                                             'movement_reactions']])
                         for p in team.index if team.loc[p, 'ST_dummy'] == 1)

# Set priorities for the multi-objective functions
model2.setObjectiveN(obj_def_mid, index=4, priority=4, name='MaximizeDefMidSkills')
model2.setObjectiveN(obj_att_mid, index=3, priority=3, name='MaximizeAttSkills')
model2.setObjectiveN(obj_striker, index=2, priority=2, name='MaximizeStrikerSkills')
model2.setObjectiveN(obj_def, index=1, priority=1, name='MaximizeDefenderSkills')
model2.setObjectiveN(obj_goalkeeper, index=0, priority=0, name='MaximizeGoalKeeperSkills')

model2.modelSense = GRB.MAXIMIZE

In [11]:
# Constraints

# Ensure each player is selected at most once
for p in team.index:
    model2.addConstr(sum(select[p, pos] for pos in all_positions if team.loc[p, f'{pos}_dummy'] == 1) <= 1, f"Player_{p}_Once")

# Specific number of players for each position

# One Goalkeeper (GK)
model2.addConstr(sum(select[p, 'GK'] for p in team.index if team.loc[p, 'GK_dummy'] == 1) == 1, "one_goalkeeper")

# Two Centre Backs (CB)
model2.addConstr(sum(select[p, 'CB'] for p in team.index if team.loc[p, 'CB_dummy'] == 1) == 2, "two_centre_backs")

# One Left Back (LB)
model2.addConstr(sum(select[p, 'LB'] for p in team.index if team.loc[p, 'LB_dummy'] == 1) == 1, "one_left_back")

# One Right Back (RB)
model2.addConstr(sum(select[p, 'RB'] for p in team.index if team.loc[p, 'RB_dummy'] == 1) == 1, "one_right_back")

# Two Defensive Midfielders (CDM)
model2.addConstr(sum(select[p, 'CDM'] for p in team.index if team.loc[p, 'CDM_dummy'] == 1) == 2, "two_defensive_midfielders")

# One Attacking Midfielder (CAM)
model2.addConstr(sum(select[p, 'CAM'] for p in team.index if team.loc[p, 'CAM_dummy'] == 1) == 1, "one_attacking_midfielder")

# One Left Winger (LW) or Left Midfielder (LM)
model2.addConstr(sum(select[p, 'LM'] for p in team.index if team.loc[p, 'LM_dummy'] == 1) + sum(select[p, 'LW'] for p in team.index if team.loc[p, 'LW_dummy'] == 1) == 1, "one_left_wing")

# One Right Midfielder (RM)
model2.addConstr(sum(select[p, 'RM'] for p in team.index if team.loc[p, 'RM_dummy'] == 1)== 1, "one_right_midfielders")

# One Striker (ST)
model2.addConstr(sum(select[p, 'ST'] for p in team.index if team.loc[p, 'ST_dummy'] == 1) == 1, "one_striker")

# Ensure total team size is 11
model2.addConstr(sum(select[p, pos] for p in team.index for pos in all_positions if team.loc[p, f'{pos}_dummy'] == 1) == 11, "eleven_players")

<gurobi.Constr *Awaiting Model Update*>

In [12]:
# Optimize the model
model2.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 33 rows, 253 columns and 125 nonzeros
Model fingerprint: 0x17f51b79
Variable types: 0 continuous, 253 integer (253 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+02, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 5 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 16 rows and 7 columns
Presolve time: 0.00s
Presolved: 17 rows and 246 columns
---------------------------------------------------------------------------

Multi-objectiv

In [13]:
if model2.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    print("Selected Players:")
    for p in team.index:
        for pos in all_positions:
            if team.loc[p, f'{pos}_dummy'] == 1 and select[p, pos].X > 0.5:
                print(f"Player Name: {team.loc[p, 'long_name']}, Position: {pos}")

elif model2.status == GRB.INFEASIBLE:
    print("The model is infeasible. No solution exists under the given constraints.")
elif model2.status == GRB.UNBOUNDED:
    print("The model is unbounded.")
else:
    print("Optimization ended with status:", model2.status)

Optimal solution found.
Selected Players:
Player Name: Chris Smalling, Position: CB
Player Name: Daley Blind, Position: LB
Player Name: Ander Herrera Agüera, Position: CDM
Player Name: Paul Pogba, Position: CDM
Player Name: Juan Manuel Mata García, Position: RM
Player Name: Ashley Young, Position: RB
Player Name: Zlatan Ibrahimovi_, Position: ST
Player Name: Wayne Mark Rooney, Position: CAM
Player Name: Cristiano Ronaldo dos Santos Aveiro, Position: LW
Player Name: David De Gea Quintana, Position: GK
Player Name: Giorgio Chiellini, Position: CB


## Part 3. Starting Line-Up Squad Selection (For 4-3-3 Formation)

In [14]:
# Create optimization model
model3 = gb.Model("Football Team Selection 4-3-3")

# Decision variables: whether to select a player for a specific position
select = model3.addVars([(p, pos) for p in team.index for pos in all_positions], vtype=GRB.BINARY, name="select")

In [15]:
# Objective functions
# Midfielders
obj_mid = gb.quicksum(select[p, pos] * sum(team.loc[p, ['passing', 'skill_ball_control', 'mentality_vision', 'mentality_positioning', 'mentality_interceptions', 'physic']])
                      for p in team.index 
                      for pos in ['CDM', 'CAM', 'LM', 'RM'] 
                      if team.loc[p, f'{pos}_dummy'] == 1)

# Wingers
obj_wingers = gb.quicksum(select[p, 'LW'] * sum(team.loc[p, ['pace', 'dribbling', 'attacking_crossing', 'shooting', 'movement_reactions', 'movement_agility']])
                         for p in team.index 
                         if team.loc[p, 'LW_dummy'] == 1)

# Striker
obj_striker = gb.quicksum(select[p, 'ST'] * sum(team.loc[p, ['shooting', 'attacking_finishing', 'attacking_heading_accuracy', 'skill_ball_control', 'mentality_positioning']])
                         for p in team.index if team.loc[p, 'ST_dummy'] == 1)

# Defenders
obj_defenders = gb.quicksum(select[p, pos] * sum(team.loc[p, ['defending', 'power_strength', 'movement_reactions', 'defending_standing_tackle', 'defending_sliding_tackle']])
                           for p in team.index 
                           for pos in ['CB', 'LB', 'RB'] 
                           if team.loc[p, f'{pos}_dummy'] == 1)

# Goalkeeper
obj_goalkeeper = gb.quicksum(select[p, 'GK'] * sum(team.loc[p, ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_kicking']])
                            for p in team.index if team.loc[p, 'GK_dummy'] == 1)

# Set objective functions with priorities
model3.setObjectiveN(obj_mid, index=4, priority=4, name='MaximizeMidfielderSkills')
model3.setObjectiveN(obj_wingers, index=3, priority=3, name='MaximizeWingerSkills')
model3.setObjectiveN(obj_striker, index=2, priority=2, name='MaximizeStrikerSkills')
model3.setObjectiveN(obj_defenders, index=1, priority=1, name='MaximizeDefenderSkills')
model3.setObjectiveN(obj_goalkeeper, index=0, priority=0, name='MaximizeGoalKeeperSkills')

model3.modelSense = GRB.MAXIMIZE

In [16]:
# Constraints
# Ensure each player is selected for at most one position
for p in team.index:
    model3.addConstr(sum(select[p, pos] for pos in all_positions if team.loc[p, f'{pos}_dummy'] == 1) <= 1, f"Player_{p}_Once")

# Position-specific constraints
# One Goalkeeper (GK)
model3.addConstr(sum(select[p, 'GK'] for p in team.index if team.loc[p, 'GK_dummy'] == 1) == 1, "one_goalkeeper")
# Four Defenders (CB, LB, RB)
model3.addConstr(sum(select[p, 'CB'] for p in team.index if team.loc[p, 'CB_dummy'] == 1) == 2, "two_centre_backs")
model3.addConstr(sum(select[p, 'LB'] for p in team.index if team.loc[p, 'LB_dummy'] == 1) == 1, "one_left_back")
model3.addConstr(sum(select[p, 'RB'] for p in team.index if team.loc[p, 'RB_dummy'] == 1) == 1, "one_right_back")

# Two Strikers (ST)
model3.addConstr(sum(select[p, 'ST'] for p in team.index if team.loc[p, 'ST_dummy'] == 1) == 2, "two_strikers")
# Ensure total team size is 11
model3.addConstr(sum(select[p, pos] for p in team.index for pos in all_positions if team.loc[p, f'{pos}_dummy'] == 1) == 11, "eleven_players")

# Three Midfielders (CDM, CAM, LM/RM)
# One Defensive Midfielder (CDM)
model3.addConstr(sum(select[p, 'CDM'] for p in team.index if team.loc[p, 'CDM_dummy'] == 1) == 1, "one_defensive_midfielder")

# One Attacking Midfielder (CAM)
model3.addConstr(sum(select[p, 'CAM'] for p in team.index if team.loc[p, 'CAM_dummy'] == 1) == 1, "one_attacking_midfielder")

# One Left or Right Midfielder (LM or RM)
model3.addConstr(sum(select[p, 'LM'] for p in team.index if team.loc[p, 'LM_dummy'] == 1) + 
                sum(select[p, 'RM'] for p in team.index if team.loc[p, 'RM_dummy'] == 1) == 1, "one_left_or_right_midfielder")

# Optimize the model
model3.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 32 rows, 253 columns and 123 nonzeros
Model fingerprint: 0x235c72f9
Variable types: 0 continuous, 253 integer (253 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 5 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 12 rows and 3 columns
Presolve time: 0.00s
Presolved: 20 rows and 250 columns
---------------------------------------------------------------------------

Multi-objectiv

In [17]:
if model3.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    print("Selected Players:")
    for p in team.index:
        for pos in all_positions:
            if team.loc[p, f'{pos}_dummy'] == 1 and select[p, pos].X > 0.5:
                print(f"Player Name: {team.loc[p, 'long_name']}, Position: {pos}")

elif model3.status == GRB.INFEASIBLE:
    print("The model is infeasible. No solution exists under the given constraints.")
elif model3.status == GRB.UNBOUNDED:
    print("The model is unbounded.")
else:
    print("Optimization ended with status:", model3.status)

Optimal solution found.
Selected Players:
Player Name: Faustino Marcos Alberto Rojo, Position: LB
Player Name: Chris Smalling, Position: CB
Player Name: Ander Herrera Agüera, Position: CDM
Player Name: Paul Pogba, Position: CAM
Player Name: Juan Manuel Mata García, Position: RM
Player Name: Ashley Young, Position: RB
Player Name: Zlatan Ibrahimovi_, Position: ST
Player Name: Cristiano Ronaldo dos Santos Aveiro, Position: LW
Player Name: David De Gea Quintana, Position: GK
Player Name: Giorgio Chiellini, Position: CB
Player Name: Kylian Mbappé Lottin, Position: ST
